In [ ]:
import numpy as np
import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_hub as hub

from keras.applications import xception

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pathlib
data_train = pathlib.Path('/content/gdrive/My Drive/Capstone/train')
data_test = pathlib.Path('/content/gdrive/My Drive/Capstone/test')

In [ ]:
#preprocessing data
from keras.preprocessing.image import ImageDataGenerator
train_datagen  = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    data_train,
    target_size=(150,150),
    class_mode='categorical',
    batch_size=126
)

test_generator = test_datagen.flow_from_directory(
    data_test,
    target_size=(150,150),
    class_mode='categorical',
    batch_size=126
)

Found 300 images belonging to 3 classes.
Found 15 images belonging to 3 classes.


In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/tensorflow/keras-applications/xception/xception_weights_tf_dim_ordering_tf_kernels_notop.h5

--2023-05-21 11:23:23--  https://storage.googleapis.com/tensorflow/keras-applications/xception/xception_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.218.128, 108.177.12.128, 108.177.13.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.218.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 83683744 (80M) [application/octet-stream]
Saving to: ‘xception_weights_tf_dim_ordering_tf_kernels_notop.h5.2’

xception_weights_tf 100%[===================>]  79.81M   188MB/s    in 0.4s    

2023-05-21 11:23:24 (188 MB/s) - ‘xception_weights_tf_dim_ordering_tf_kernels_notop.h5.2’ saved [83683744/83683744]



In [ ]:
base_model = xception.Xception(
    weights='imagenet',
    include_top=False,
    input_shape=(150,150,3),
    pooling='avg'
)

83683744/83683744 [==============================] - 1s 0us/step


In [ ]:
for layer in base_model.layers:
  layer.trainable = False

In [ ]:
last_layer = base_model.get_layer('conv2d_12')
print('last lasyer output shape :', last_layer.output_shape)
last_output = last_layer.output

last lasyer output shape : (None, 36, 36, 128)


In [ ]:
from tensorflow.keras import Model
from tensorflow.keras import layers

x = layers.Flatten()(last_output)
x = layers.Dense(128, activation = 'relu')(x)
x = layers.Dense(3, activation = 'softmax')(x)

model = Model(base_model.input, x)

In [ ]:
model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [ ]:
history = model.fit(
    train_generator,
    epochs = 100
)

Epoch 1/100
3/3 [==============================] - 6s 2s/step - loss: 26.9972 - accuracy: 0.5633
Epoch 2/100
3/3 [==============================] - 8s 2s/step - loss: 49.5461 - accuracy: 0.4633
Epoch 3/100
3/3 [==============================] - 6s 2s/step - loss: 61.5939 - accuracy: 0.3967
Epoch 4/100
3/3 [==============================] - 6s 2s/step - loss: 62.5917 - accuracy: 0.4900
Epoch 5/100
3/3 [==============================] - 7s 2s/step - loss: 70.3700 - accuracy: 0.4600
Epoch 6/100
3/3 [==============================] - 6s 2s/step - loss: 66.3679 - accuracy: 0.4033
Epoch 7/100
3/3 [==============================] - 8s 3s/step - loss: 52.0364 - accuracy: 0.5867
Epoch 8/100
3/3 [==============================] - 6s 2s/step - loss: 43.4254 - accuracy: 0.4733
Epoch 9/100
3/3 [==============================] - 6s 2s/step - loss: 36.9505 - accuracy: 0.4767
Epoch 10/100
3/3 [==============================] - 6s 1s/step - loss: 41.9114 - accuracy: 0.4800
Epoch 11/100
3/3 [===========